In [106]:
using StatsBase

# Particle Ensembles and other Types

In [107]:
AbstractHiddenState = Union{Number, AbstractVector{T} where T<:Number}
abstract type AbstractFilterRepresentation{T<:AbstractHiddenState} end;

abstract type ParticleRepresentation{T<:AbstractHiddenState} <: AbstractFilterRepresentation{T} end

abstract type UnweightedParticleRepresentation{T<:AbstractHiddenState} <: ParticleRepresentation{T} end

In [108]:
VectorHiddenState = Union{Float64, AbstractVector{T} where T<:Float64};

In [109]:
VectorHiddenState <: AbstractHiddenState

true

In [110]:
mutable struct UnweightedParticleEnsemble{T<:AbstractHiddenState} <: UnweightedParticleRepresentation{T}
    positions::AbstractVector{T}
    size::Int
    
    UnweightedParticleEnsemble(positions::AbstractVector{T}, size::Int) where T<:AbstractHiddenState = 
        if length(positions) != size
            if length(positions) == 1
                new{T}(fill(positions[1], size), size)
            else
                error("ERROR: number of particle positions does not equal ensemble size.")
            end
        else
            new{T}(positions, size)
        end
end;

In [111]:
mutable struct WeightedParticleEnsemble{T<:AbstractHiddenState} <: ParticleRepresentation{T}
    positions::AbstractVector{T}
    weights::StatsBase.ProbabilityWeights
    size::Int
    
    WeightedParticleEnsemble(positions::AbstractVector{T}, weights::StatsBase.ProbabilityWeights, size::Int) where T<:AbstractHiddenState = 
        if !length(positions) == length(weights) == size
            if length(positions) == 1
                new{T}(fill(positions[1], size), fill(1., size), size)
            else
                error("ERROR: number of particle positions does not equal number of weights or ensemble size.")
            end
        else
            new{T}(positions, weights, size)
        end
end;

In [112]:
abstract type AbstractGainEquation{T<:AbstractHiddenState} end;
struct EmptyGainEquation{T<:AbstractHiddenState} <: AbstractGainEquation{T} end;

In [113]:
mutable struct FPFEnsemble{T<:AbstractHiddenState} <: UnweightedParticleRepresentation{T}
    positions::AbstractVector{T}
    size::Int
    eq::AbstractGainEquation{T}
    FPFEnsemble(positions::AbstractVector{T}, size::Int) where T<:AbstractHiddenState = 
        if length(positions) != size
            if length(positions) == 1
                new{T}(fill(positions[1], size), size, EmptyGainEquation{T}())
            else
                error("ERROR: number of particle positions does not equal ensemble size.")
            end
        else
            new{T}(positions, size, EmptyGainEquation{T}())
        end
end;

In [114]:
ens = FPFEnsemble(randn(10000), 10000);

# Fast mapping of functions

In [83]:
function Map(f::T, A::AbstractArray) where T<:Function 
    map(f, A)
end
function Map(F::NTuple{N, Function}, x::T) where {N, T<:Number} 
    [f(x) for f in F]
end;
function Map(F::AbstractArray{Function}, x::T) where {N, T<:Number} 
    [f(x) for f in F]
end;
function Map(F::AbstractArray{U}, x::T) where {N, T<:Number, U<:Function}
    [f(x) for f in F]
end;
function Map(F::NTuple{N, Function}, A::AbstractArray; output_shape=2) where N
    if output_shape == 1 
        try
            collect(Map.(F, Ref(A)))
        catch
            error("ERROR: functions cannot be applied at first level. Call with output_shape=2.")
        end
    elseif output_shape == 2
        try
            [f(A) for f in F]
        catch
            collect(Map.(Ref(F), A))
        end
    else
        error("ERROR: Invalid output_shape parameter. Must be either 1 or 2.") 
    end
end;
function Map(F::AbstractArray{T}, A::AbstractArray; output_shape=2) where {N, T<:Function}
    if output_shape == 1 
        try
            collect(Map.(F, Ref(A)))
        catch
            error("ERROR: functions cannot be applied at first level. Call with output_shape=2.")
        end
    elseif output_shape == 2
        try
            [f(A) for f in F]
        catch
            collect(Map.(Ref(F), A))
        end
    else
        error("ERROR: Invalid output_shape parameter. Must be either 1 or 2.") 
    end
end;

In [2]:
f(x::Float64) = x
g(x::Float64) = x^2
h(x::Float64) = x^3;

In [3]:
function ff(n::Int)
    f(x::Float64) = x^n
    return f::Function
end;
F = Array{Function, 1}(undef, 100)
for n in 1:length(F)
    F[n] = ff(n)
end;
Ftuple = Tuple(F);

In [4]:
vec = randn(10000);

In [5]:
@time collect(map.(f, Ref(vec)));

  0.074441 seconds (173.29 k allocations: 9.727 MiB)


In [73]:
@time [map(fun, vec) for fun in F]
@time [Map(fun, vec) for fun in F]
@time Map(F, vec);
@time Map(Ftuple, vec);

  0.066874 seconds (59.28 k allocations: 10.619 MiB)
  0.074483 seconds (59.33 k allocations: 10.622 MiB, 6.12% gc time)
  0.071172 seconds (2.04 M allocations: 39.818 MiB, 4.98% gc time)
  0.030071 seconds (10.01 k allocations: 8.699 MiB, 8.67% gc time)


In [7]:
@time Map(Ftuple, vec, output_shape=1)
@time Map(Ftuple, vec, output_shape=2);

  0.047928 seconds (8.96 k allocations: 8.122 MiB)
  0.029995 seconds (10.02 k allocations: 8.698 MiB)


In [8]:
ff(x::Array{Float64,1}) = x[1]*x[2]
gg(x::Array{Float64,1}) = x[1]+x[2]
hh(x::Array{Float64,1}) = x[1]-x[2];

In [95]:
@time Map((ff,gg,hh), [[randn(2) for i in 1:100] for j in 1:100], output_shape=2);

  0.091537 seconds (193.75 k allocations: 10.463 MiB, 6.60% gc time)


In [61]:
[[randn(2) for i in 1:2] for j in 1:2]

2-element Array{Array{Array{Float64,1},1},1}:
 [[0.596676, -0.708981], [1.00465, -1.27132]] 
 [[0.495679, 0.334232], [-0.551361, -1.21584]]

## Evaluate functions on particle ensembles

In [115]:
function Map(f::U, ensemble::T) where {U<:Function, T<:UnweightedParticleRepresentation{S}} where S<:AbstractHiddenState
    if !hasmethod(f, tuple(S))
        error("ERROR: this function cannot be evaluated for this ensemble of particles.")
    end
    Map(f, ensemble.positions)
end;

In [116]:
function Map(F::NTuple{N, U}, ensemble::T) where {N, U<:Function, T<:UnweightedParticleRepresentation{S}} where S<:AbstractHiddenState
    if !hasmethod(f, tuple(S))
        error("ERROR: this function cannot be evaluated for this ensemble of particles.")
    end
    Map(F, ensemble.positions)
end;

In [121]:
@time [map(fun, ens.positions) for fun in F]
@time [Map(fun, ens) for fun in F]
@time Map(Ftuple, ens);

  0.079676 seconds (59.38 k allocations: 10.623 MiB, 4.58% gc time)
  0.076187 seconds (59.57 k allocations: 10.626 MiB)
  0.033524 seconds (10.02 k allocations: 8.700 MiB, 14.57% gc time)


In [126]:
hasmethod(Map, Tuple(U,T) where {U<:Function, T<:UnweightedParticleRepresentation{S}} where S<:AbstractHiddenState)

MethodError: MethodError: no method matching Tuple(::TypeVar, ::TypeVar)
Closest candidates are:
  Tuple(::Any) where T<:Tuple at tuple.jl:246

# Fast computation of expectations

In [58]:
function Expectation(A::AbstractArray{T}) where T<:VectorHiddenState
    StatsBase.mean(A)
end;
function Expectation(f::U, A::AbstractArray{T}) where {U<:Function, T<:VectorHiddenState}
    StatsBase.mean(f, A)
end;

In [60]:
function Expectation(ensemble::T) where T<:UnweightedParticleRepresentation{S} where S<:VectorHiddenState
    Expectation(ensemble.positions)
end;

In [83]:
ens = FPFEnsemble(randn(10000), 10000);
@time Expectation(ens.positions);
@time Expectation(ens);

  0.000018 seconds (5 allocations: 176 bytes)
  0.000017 seconds (5 allocations: 176 bytes)


In [17]:
function StatsBase.mean(f::Function, ensemble::T) where T<:UnweightedParticleRepresentation{S} where S <: AbstractHiddenState
    if !hasmethod(f, tuple(S))
        error("ERROR: this function cannot be evaluated for this ensemble of particles.")
    end
    StatsBase.mean(f, ensemble.positions)
end

In [18]:
ens = FPFEnsemble(randn(10000), 10000);

In [55]:
mean([randn(10) for i in 1:100])

10-element Array{Float64,1}:
  0.2044916195636541   
 -0.025351193565437892 
  0.004773154179286381 
  0.04238315331875718  
  0.05628195966075676  
 -0.11476142893607554  
 -0.11800978757255086  
  0.0021916009795702983
  0.032040668177654344 
 -0.1873175611500785   

In [389]:
isa(ff(1), Function)

true

In [527]:
F = Array{Function, 1}(undef, 100)
for n in 1:length(F)
    F[n] = ff(n)
end;

In [528]:
typeof(F)

Array{Function,1}

In [529]:
function StatsBase.mean(F::AbstractArray{Function}, A::Any)
    [StatsBase.mean(f, A) for f in F]
end
function Base.map(F::AbstractArray{Function}, A::AbstractArray)
    [Base.map(f, A) for f in F]
end
function mymap(F::NTuple{N, Function}, A::AbstractArray) where N
    #res = [Base.map(f, A) for f in F]
    #[[f(a) for f in F] for a in A]
    collect(map.(F, Ref(A)))
end;

In [530]:
vec = randn(10000);

In [531]:
Ftuple = Tuple(F);

In [533]:
@time [map(g, vec) for i in 1:100];
@time res=map(F, vec);
@time res=mymap(Ftuple, vec);

  0.047211 seconds (59.04 k allocations: 10.614 MiB)
  0.037903 seconds (307 allocations: 7.641 MiB, 21.18% gc time)
  0.030579 seconds (306 allocations: 7.642 MiB)


In [9]:
function Map(F::NTuple{N, Function}, A::AbstractArray) where N
    collect(map.(F, Ref(A)))
end;

ErrorException: cannot define function Map; it already has a value

In [544]:
function Map(F::AbstractArray{Function}, A::AbstractArray) where N
    #res = [Base.map(f, A) for f in F]
    #[[f(a) for f in F] for a in A]
    collect(map.(Tuple(F), Ref(A)))
end;

ErrorException: cannot define function Map; it already has a value

In [ ]:
function Update!(filter_state::F, object::M) where {F<:AbstractFilterRepresentation, M<:Union{AbstractHiddenStateModel, AbstractObservation}}
    object(filter_state)

In [ ]:
function Update!(ensemble::FPFEnsemble)

In [543]:
methods(mean)

# 10 methods for generic function "mean":
[1] mean(r::AbstractRange{#s623} where #s623<:Real) in Statistics at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.1/Statistics/src/Statistics.jl:138
[2] mean(A::AbstractArray{T,N} where N, w::AbstractWeights{W,T,V} where V<:AbstractArray{T,1} where T<:Real, dims::Int64) where {T<:Number, W<:Real} in StatsBase at deprecated.jl:55
[3] mean(F::AbstractArray{Function,N} where N, A) in Main at In[529]:2
[4] mean(A::AbstractArray; dims) in Statistics at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.1/Statistics/src/Statistics.jl:132
[5] mean(A::AbstractArray, w::AbstractWeights; dims) in StatsBase at /Users/simone/.julia/packages/StatsBase/rTkaz/src/weights.jl:469
[6] mean(ensemble::T) where T<:(UnweightedParticleRepresentation{S} where S<:Union{Float64, Array{N,1} where N<:Float64}) in Main at In[15]:2
[7] mean(f::Function, ensemble::T) where {S<:Union{Number, AbstractArray{T,1} where T<:Number}, T<:UnweightedParticleRepresentation{S<:Union{Number, AbstractArray{T,1} where T<:Number}}} in Main at In[17]:2
[8] mean(itr) in Statistics at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.1/Statistics/src/Statistics.jl:42
[9] mean(f, A::AbstractArray) in Statistics at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.1/Statistics/src/Statistics.jl:76
[10] mean(f, itr) in Statistics at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.1/Statistics/src/Statistics.jl:58

In [175]:
function _pairwise!(r::AbstractMatrix, f::Function,
                    a::AbstractVector, b::AbstractVector)
    na = length(a)
    nb = length(b)
    size(r) == (na, nb) || throw(DimensionMismatch("Incorrect size of r."))
    @inbounds @simd for j = 1:nb
        @simd for i = 1:na
            r[i, j] = f(a[i], b[j])
        end
    end
    r
end

_pairwise! (generic function with 1 method)

In [115]:
res = zeros(Float64, 2000, 5000)
a = randn(2000)
b = randn(5000)

fun(x::Float64, y::Float64) = x*y

fun (generic function with 1 method)

In [119]:
@time _pairwise!(res, fun, a, b);

  0.007388 seconds (4 allocations: 160 bytes)


In [190]:
applyfun(f::Function, x::Float64) = f(x);
applyfun2(x::Float64, f::Function) = f(x);

In [193]:
res = zeros(Float64, 3, 10000)
@time _pairwise!(res, applyfun, F, ens.positions);

  0.003861 seconds (90.00 k allocations: 1.373 MiB)


In [178]:
@time map(F, ens.positions);

  0.000126 seconds (16 allocations: 234.969 KiB)


In [196]:
res = zeros(Float64, 10000, 3)
@time _pairwise!(res, applyfun2, ens.positions, F);

  0.003208 seconds (90.00 k allocations: 1.373 MiB)


In [189]:
res

10000×3 Array{Float64,2}:
 -0.437412    0.191329    -0.0836898 
 -0.00981145  9.62645e-5  -9.44494e-7
 -0.862003    0.743049    -0.640511  
  0.0448808   0.00201429   9.04028e-5
  1.04089     1.08344      1.12774   
 -0.761191    0.579412    -0.441044  
  0.838916    0.70378      0.590412  
 -0.213966    0.0457813   -0.00979561
 -0.259926    0.0675616   -0.017561  
  0.170285    0.028997     0.00493776
 -1.06506     1.13434     -1.20814   
  0.142437    0.0202883    0.00288981
  0.646614    0.418109     0.270355  
  ⋮                                  
  0.498481    0.248483     0.123864  
  0.662371    0.438735     0.290605  
 -1.01093     1.02198     -1.03316   
 -0.566238    0.320626    -0.181551  
 -0.985352    0.97092     -0.956698  
 -0.388485    0.15092     -0.0586302 
  0.475049    0.225672     0.107205  
  0.926799    0.858957     0.79608   
  0.776673    0.603222     0.468506  
  1.38527     1.91897      2.65829   
 -0.535231    0.286473    -0.153329  
  0.562469    0.316371  